In [1]:
import tensorflow as tf
import os

In [2]:
train_dir = "/Users/ttdat/Documents/Do_An_Co_So/code/code_final/data_split/train"
test_dir = "/Users/ttdat/Documents/Do_An_Co_So/code/code_final/data_split/test"
img_size = (160, 160)
batch_size = 32

In [3]:
# Load ảnh thành dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'  # vì softmax
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

Found 791 files belonging to 51 classes.
Found 391 files belonging to 51 classes.


2025-05-13 20:47:58.247951: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-05-13 20:47:58.249380: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-13 20:47:58.249385: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-05-13 20:47:58.249418: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-13 20:47:58.249431: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Tối ưu hiệu suất
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [5]:
# Mô hình MobileNetV2 + phân lớp
base_model = tf.keras.applications.MobileNetV2(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
output = tf.keras.layers.Dense(train_ds.element_spec[1].shape[-1], activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=output)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
# Huấn luyện
model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10


2025-05-13 20:47:59.801416: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 201ms/step - accuracy: 0.2657 - loss: 3.5367 - val_accuracy: 0.5115 - val_loss: 1.7274
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.5477 - loss: 1.3673 - val_accuracy: 0.5601 - val_loss: 1.2847
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.6003 - loss: 0.9998 - val_accuracy: 0.5422 - val_loss: 1.0982
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.6277 - loss: 0.8795 - val_accuracy: 0.5601 - val_loss: 1.0943
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.6142 - loss: 0.8557 - val_accuracy: 0.5371 - val_loss: 1.0141
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.6734 - loss: 0.7131 - val_accuracy: 0.5064 - val_loss: 0.9991
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.6771 - loss: 0.6923 - val_accuracy: 0.5601 - val_loss: 1.1270
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.6503 - loss: 0.7039 - val_accuracy: 0.5371 - val_loss: 1